<a href="https://colab.research.google.com/github/grace7724/meeting-iaq-analysis/blob/main/Effects_of_Meeting_Type_on_Indoor_Environmental_Conditions_in_Shared_Spaces_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Notebook Overview
# Effects of Meeting Type on Indoor Environmental Conditions in Shared Spaces - Analysis Notebook

This notebook reproduces the analysis for the paper:

> **Effects of Meeting Type on Indoor Environmental Conditions in Shared Spaces**

CE 6070: Smart and Healthy Buildings, Group 6

It performs the following steps:

1. **Data acquisition**
   - Pull sensor time series from the University of Virginia (UVA) Living Link Lab InfluxDB.
   - Retrieve Google Calendar events for three conference rooms at the UVA in Olsson Hall.
2. **Data cleaning and canonicalization**
   - Standardize sensor variable names and units.
   - Normalize meeting titles and assign them to six categories.
3. **Filtering unrealized meetings**
   - Use CO₂ patterns to identify which scheduled meetings actually occurred.
4. **Feature engineering**
   - Compute pre/during/post IAQ summaries and impact features.
5. **Merging features with calendar metadata**
   - Create final analysis tables `X_all` and `Y_all`.
6. **Results**
   - Peak impact by meeting type.
   - Post-meeting recovery times.
   - Duration effects.
   - Time-course traces for CO₂ and VOC.
7. **Exports**
   - Save intermediate tables, final analysis tables, and figures to disk.

> **Note:** The InfluxDB and calendar sections rely on UVA Link Lab data.  
> If you already have the exported CSV files, you can start from the “Load Raw Data” section.

# 1. Setup and Imports

In [ ]:
# 1.0 Setup
# ----------
# Core imports, plotting style, and global configuration.

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import os
import re
from dataclasses import dataclass
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from concurrent.futures import ThreadPoolExecutor, as_completed

# Influx and calendar-related imports
from __future__ import annotations
import pytz
import requests
from icalendar import Calendar
import recurring_ical_events

# InfluxDB client (used for raw sensor pull)
from influxdb import InfluxDBClient

# Global plotting style
sns.set_theme(style="whitegrid")

# Directories for outputs
DATA_DIR = Path("data")
CACHE_DIR = Path("cache")
FIG_DIR = Path("figures")
STATS_DIR = Path("stats")
EXPORT_DIR = Path("exports")

for d in [DATA_DIR, CACHE_DIR, FIG_DIR, STATS_DIR, EXPORT_DIR]:
    d.mkdir(exist_ok=True)

def savefig(name: str, dpi: int = 300) -> None:
    """
    Save the current matplotlib figure into figures/ as PNG.

    Parameters
    ----------
    name : str
        Base file name (without extension).
    dpi : int
        Resolution for saved PNG.
    """
    fname = FIG_DIR / f"{name}.png"
    plt.tight_layout()
    plt.savefig(fname, dpi=dpi, bbox_inches="tight")
    plt.close()
    print(f"Saved figure → {fname}")

print("Setup complete.")

Setup complete.


# 2. Data Acquisition

### 2.1 Sensor Data from InfluxDB

This section pulls **1-minute aggregated sensor data** for three rooms in Olsson Hall Link Lab from the Link Lab InfluxDB and writes:
- A per-room CSV for each room.
- A stacked multi-room CSV:
  `olsson_multiroom_stack_1min_2018-01-01_to_2025-11-01.csv`

If that CSV already exists, you can skip running this block.

In [ ]:
# InfluxDB connection configuration


# Create InfluxDB client (SSL)
client = InfluxDBClient(HOST, PORT, USERNAME, PASSWORD, DATABASE,
                        ssl=True, verify_ssl=True)

def result_to_dataframe(result) -> pd.DataFrame:
    """
    Convert an InfluxDB query result into a tidy pandas DataFrame with a
    UTC DatetimeIndex named 'time'.

    Handles empty results safely.
    """
    if not result:
        return pd.DataFrame()
    points = list(result.get_points())
    if not points:
        return pd.DataFrame()

    df = pd.DataFrame(points)

    # Normalize time column to UTC Timestamp
    df["time"] = pd.to_datetime(df["time"], errors="coerce", utc=True)
    df = df.dropna(subset=["time"])
    return df

# Rooms to pull
rooms = [
    "211 Olsson",
    "217 Olsson",
    "225 Olsson",
]

# Mapping from friendly names to Influx measurement names
measurements_map = {
    "co2_ppm": "co2_ppm",
    "voc_ppb": "voc_ppb",
    "pm2.5_g/m3": 'pm2.5_g/m3',
    "awair_score": "awair_score",
    "Humidity_%": "Humidity_%",
    "Temperature_°C": "Temperature_°C",
    "Illumination_lx": "Illumination_lx",
    "spl_a": "spl_a",
    "vector_4d_count": "vector_4d_count",
    "occupancy_count": "occupancy_count",
    "power_w": "power_w",
    "energy_kWh": "energy_kWh",
    "current_a": "current_a",
    "power_factor": "power_factor",
    "Contact": "Contact",
    "PIR Status": "PIR Status",
}

# Time window for queries (matches downstream analysis)
start_time = datetime(2018, 1, 1)
end_time = datetime(2025, 11, 1)

def influx_time_iso(dt: datetime) -> str:
    """
    Return UTC ISO8601 string with trailing 'Z' to use in InfluxQL time filters.
    """
    return pd.Timestamp(dt, tz="UTC").isoformat().replace("+00:00", "Z")

def fetch_sensor(meas_influx: str, room: str) -> pd.DataFrame:
    """
    Query one measurement for one room with 1-minute aggregation.

    Returns
    -------
    DataFrame with columns: ['time', <friendly_name>]
    """
    q = f"""
    SELECT mean("value") AS "value"
    FROM "{meas_influx}"
    WHERE "location_specific" = '{room}'
    AND time >= '{influx_time_iso(start_time)}'
    AND time < '{influx_time_iso(end_time)}'
    GROUP BY time(1m) fill(linear)
    """

    try:
        result = client.query(q)
        df = result_to_dataframe(result)
        return df
    except Exception as e:
        print(f"{meas_influx} ({room}) failed: {e}")
        return pd.DataFrame()

def merge_by_time(dfs: list[pd.DataFrame], names: list[str]) -> pd.DataFrame:
    """
    ASOF-merge multiple single-series dataframes on 'time' within 1-minute
    tolerance, then apply light gap filling.
    """
    if not dfs:
        return pd.DataFrame()

    prepped = []
    for df, friendly in zip(dfs, names):
        if df is None or df.empty:
            continue
        d = df.copy()
        if "value" in d.columns:
            d.rename(columns={"value": friendly}, inplace=True)

        keep = ["time", friendly] if friendly in d.columns else ["time"]
        d = d[keep].sort_values("time")
        prepped.append(d)

    if not prepped:
        return pd.DataFrame()

    merged = prepped[0]
    for d in prepped[1:]:
        merged = pd.merge_asof(
            merged.sort_values("time"),
            d.sort_values("time"),
            on="time",
            tolerance=pd.Timedelta("1min"),
            direction="nearest",
        )

    merged = merged.sort_values("time").set_index("time")

    # Light gap filling to reduce sparsity
    merged = merged.ffill(limit=5).bfill(limit=5).interpolate(
        limit=5, limit_direction="both"
    )

    merged = merged.reset_index()
    return merged

def slugify_room(room: str) -> str:
    """Simple room slug for filenames."""
    return room.lower().replace(" ", "").replace("/", "-")

# Pull all rooms and write per-room and stacked sensor CSVs
all_rooms_long = []

for room in rooms:
    print(f"\n=== Room: {room} ===")

    dfs, names = [], []

    # Parallelize per-measurement queries for each room (this speeds the process up)
    with ThreadPoolExecutor(max_workers=6) as executor:
        future_map = {}
        for friendly, meas_influx in measurements_map.items():
            fut = executor.submit(fetch_sensor, meas_influx, room)
            future_map[fut] = (friendly, meas_influx)

        for fut in as_completed(future_map):
            friendly, meas_influx = future_map[fut]
            df = fut.result()
            if df is not None and not df.empty:
                print(f" {friendly:18s} ({meas_influx}): {len(df)} rows")
                dfs.append(df)
                names.append(friendly)
            else:
                print(f"— {friendly:18s} ({meas_influx}): no data")

    print(f"Retrieved {len(dfs)} / {len(measurements_map)} series for {room}.")

    if not dfs:
        print(f"Skipping {room}: no data found in window.")
        continue

    df_room = merge_by_time(dfs, names)
    if df_room.empty:
        print(f"Merge produced empty frame for {room}.")
        continue

    # Save wide per-room CSV
    slug = slugify_room(room)
    csv_path = f"olsson_{slug}_1min_{start_time.date()}_to_{end_time.date()}.csv"
    df_room.to_csv(csv_path, index=False)
    print(f"Saved {csv_path} ({len(df_room)} rows x {len(df_room.columns)} cols).")

    # Add 'room' column for stacked analysis
    df_long = df_room.copy()
    df_long["room"] = room
    all_rooms_long.append(df_long)

# Write stacked multi-room CSV for downstream analysis
if all_rooms_long:
    stacked = pd.concat(all_rooms_long, ignore_index=True, sort=False)
    stacked = stacked[["time", "room"] + [c for c in stacked.columns
                                          if c not in ("time", "room")]]
    stacked_path = f"olsson_multiroom_stack_1min_{start_time.date()}_to_{end_time.date()}.csv"
    stacked.to_csv(stacked_path, index=False)
    print(f"\nSaved stacked multi-room CSV to {stacked_path} ({len(stacked)} rows).")
else:
    print("\nNo stacked CSV written (no rooms returned data).")

### 2.2 Calendar Data from Google Calendar

This section:

- Retrives public ICS feeds for the three rooms.
- Expands recurring events using `recurring_ical_events`.
- Normalizes timestamps.
- Produces a single CSV: `olsson_calendars_2018-01-01_to_2025-11-01.csv`

In [ ]:
NY = pytz.timezone("America/New_York")

# Mapping from room name to Google Calendar ID
CAL_IDS = {
    "211 Olsson": "de9e37qei89hbdhe03qlq8mv40@group.calendar.google.com",
    "217 Olsson": "oljniar4mj77b1oknmk9stoad0@group.calendar.google.com",
    "225 Olsson": "fi663en0bb24fl20a875tnbup8@group.calendar.google.com",
}

def ics_url(cal_id: str) -> str:
    """Construct the public FULL ICS URL for a given calendar ID."""
    return f"https://calendar.google.com/calendar/ical/{cal_id}/public/full.ics"

# Calendar time window, matching sensor pull
start = NY.localize(datetime(2018, 1, 1))
end = NY.localize(datetime(2025, 11, 1))

def fetch_calendar(cal_id: str) -> Calendar | None:
    """
    Download and parse one ICS calendar.

    Returns
    -------
    icalendar.Calendar or None on failure.
    """
    url = ics_url(cal_id)
    r = requests.get(url, timeout=30)
    if r.status_code != 200:
        print(f" {cal_id}: HTTP {r.status_code} — is the calendar public?")
        return None
    return Calendar.from_ical(r.content)

def expand_events(cal: Calendar, start_dt: datetime, end_dt: datetime):
    """
    Expand VEVENTs over [start_dt, end_dt] including recurrences.
    """
    return recurring_ical_events.of(cal).between(start_dt, end_dt)

def to_dt(v) -> pd.Timestamp:
    """
    Convert an ical date/datetime to a tz-aware pandas Timestamp in NY tz.
    """
    ts = pd.to_datetime(v)
    if ts.tzinfo is None:
        # Treat date-only or naive datetime as local NY time
        ts = NY.localize(ts.to_pydatetime())
    else:
        ts = ts.tz_convert(NY)
    return ts

def calendar_to_df(room: str, cal_id: str) -> pd.DataFrame:
    """
    Fetch, expand, and tidy the calendar for one room.

    Returns
    -------
    DataFrame with one row per event occurrence and columns including:
    room, calendar_id, summary, description, location, start, end, etc.
    """
    cal = fetch_calendar(cal_id)
    if cal is None:
        return pd.DataFrame()

    events = []
    for ev in expand_events(cal, start, end):
        comp = ev
        dtstart = comp.get('dtstart')
        dtend = comp.get('dtend')
        if not dtstart or not dtend:
            continue

        events.append({
            "room": room,
            "calendar_id": cal_id,
            "summary": str(comp.get('summary', "")),
            "description": str(comp.get('description', "")),
            "location": str(comp.get('location', "")),
            "start": to_dt(dtstart.dt),
            "end": to_dt(dtend.dt),
            "all_day": (hasattr(dtstart.dt, "date") and not hasattr(dtstart.dt, "hour")),
            "created": pd.to_datetime(comp.get('created').dt) if comp.get('created') else pd.NaT,
            "updated": pd.to_datetime(comp.get('last-modified').dt) if comp.get('last-modified') else pd.NaT,
            "uid": str(comp.get('uid', "")),
            "organizer": str(comp.get('organizer', "")),
        })

    df = pd.DataFrame(events)
    if not df.empty:
        # Add UTC versions of start/end to align with sensor timestamps
        df["start_utc"] = df["start"].dt.tz_convert("UTC")
        df["end_utc"] = df["end"].dt.tz_convert("UTC")
        df = df.sort_values(["room", "start"])
    return df

# Pull calendars for all rooms and concatenate
dfs = []
for room, cal_id in CAL_IDS.items():
    print(f"Fetching {room} …")
    df = calendar_to_df(room, cal_id)
    print(f" {len(df)} events")
    dfs.append(df)

cal_all = pd.concat([d for d in dfs if d is not None and not d.empty],
                    ignore_index=True)
print(f"\nTotal events: {len(cal_all)}")

out_csv = f"olsson_calendars_{start.date()}_to_{end.date()}.csv"
cal_all.to_csv(out_csv, index=False)
print("Saved:", out_csv)

cal_all.head()

# 3. Load Raw Data

From this point forward, the analysis assumes:

- A calendar CSV: `olsson_calendars_2018-01-01_to_2025-11-01.csv`
- A stacked sensor CSV: `olsson_multiroom_stack_1min_2018-01-01_to_2025-11-01.csv`

If you ran the acquisition sections above, these files should already exist.

### 3.1 Calendar Data

In [ ]:
# Candidate locations for the calendar CSV
CAL_CSV_CANDIDATES = [
    "/mnt/data/olsson_calendars_2018-01-01_to_2025-11-01.csv",  # hosted env
    "cache/olsson_calendars_2018-01-01_to_2025-11-01.csv.gz",
    "olsson_calendars_2018-01-01_to_2025-11-01.csv",
]

for _p in CAL_CSV_CANDIDATES:
    if Path(_p).exists():
        CAL_PATH = _p
        break
else:
    raise FileNotFoundError(
        "Calendar CSV not found. Put it at one of:\n"
        " - /mnt/data/olsson_calendars_2018-01-01_to_2025-11-01.csv\n"
        " - cache/olsson_calendars_2018-01-01_to_2025-11-01.csv.gz\n"
        " - olsson_calendars_2018-01-01_to_2025-11-01.csv"
    )

LOCAL_TZ = "US/Eastern"

# Read and normalize calendar data
df_cal = pd.read_csv(CAL_PATH, parse_dates=["start", "end"])

for c in ("start", "end"):
    df_cal[c] = (
        pd.to_datetime(df_cal[c], utc=True, errors="coerce")
        .dt.tz_convert(LOCAL_TZ)
    )

df_cal = (
    df_cal.dropna(subset=["start", "end"])
          .sort_values("start")
          .reset_index(drop=True)
)

print("Calendar rows:", len(df_cal))
df_cal.head()

### 3.2 Sensor Data

In [ ]:
# Helper to detect whether Parquet is available for caching
def parquet_ok() -> bool:
    try:
        import pyarrow  # noqa
        return True
    except Exception:
        try:
            import fastparquet  # noqa
            return True
        except Exception:
            return False

# Raw sensor CSV produced from Influx (stacked multi-room)
SENSORS_SRC = "olsson_multiroom_stack_1min_2018-01-01_to_2025-11-01.csv"

# Cache locations (to avoid re-parsing large CSVs repeatedly)
PQ_PATH = CACHE_DIR / "sensors.parquet"
CSV_PATH = CACHE_DIR / "sensors.csv.gz"

# Canonical set of measurement names used in analysis
MEASUREMENTS = [
    "co2_ppm", "voc_ppb", "pm25_ugm3", "awair_score",
    "rh_percent", "temp_c", "Illumination_lx", "spl_a",
    "vector_4d_count", "occupancy_count",
    "power_w", "energy_kWh", "current_a",
    "power_factor", "Contact", "pir"
]

def canonicalize_sensor_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rename raw sensor column names to canonical ones used throughout analysis.
    """
    ren = {
        "pm2.5_g/m3": "pm25_ugm3",
        "pm2.5_ug/m3": "pm25_ugm3",
        "PM_2.5": "pm25_ugm3",
        "Humidity_%": "rh_percent",
        "Temperature_°C": "temp_c",
        "PIR Status": "pir",
        "spl_A": "spl_a",
        "Illuminance_lx": "Illumination_lx",
    }
    df = df.rename(columns={k: v for k, v in ren.items() if k in df.columns})
    return df

def load_sensors(force_reload: bool = False) -> pd.DataFrame:
    """
    Load sensor data with optional caching.

    Returns
    -------
    DataFrame indexed by UTC 'time' with:
    - 'room' column
    - Canonical measurement columns listed in MEASUREMENTS
    """
    # Reset cache if requested
    if force_reload:
        if PQ_PATH.exists():
            PQ_PATH.unlink()
        if CSV_PATH.exists():
            CSV_PATH.unlink()

    # 1) Use Parquet cache if available
    if PQ_PATH.exists() and parquet_ok() and not force_reload:
        df = pd.read_parquet(PQ_PATH)
        df.index = pd.to_datetime(df.index, utc=True)

    # 2) Otherwise use compressed CSV cache
    elif CSV_PATH.exists() and not force_reload:
        df = pd.read_csv(CSV_PATH, parse_dates=["time"])
        df["time"] = pd.to_datetime(df["time"], utc=True, errors="coerce")
        df = df.set_index("time")

    # 3) Otherwise load from the original sensor CSV
    else:
        if not Path(SENSORS_SRC).exists():
            raise FileNotFoundError(
                f"Sensor CSV {SENSORS_SRC} not found. "
                f"Place it in the working directory."
            )
        df = pd.read_csv(SENSORS_SRC, parse_dates=["time"])
        df["time"] = pd.to_datetime(df["time"], utc=True, errors="coerce")
        df = df.set_index("time")

        # Standardize column names
        df = canonicalize_sensor_columns(df)

        # Show which measurements are present
        missing = sorted(set(MEASUREMENTS) - set(df.columns))
        present = sorted(set(MEASUREMENTS) & set(df.columns))
        print(f"Present ({len(present)}): {present}")
        print(f"Missing ({len(missing)}): {missing}")

        keep = ["room"] + [c for c in MEASUREMENTS if c in df.columns]
        df = df[keep]

        # Cache for future runs
        if parquet_ok():
            df.to_parquet(PQ_PATH)
        else:
            df.reset_index().to_csv(
                CSV_PATH, index=False, compression="gzip"
            )

    # Ensure UTC, sorted index
    df = df.sort_index()
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC")
    else:
        df.index = df.index.tz_convert("UTC")
    df.index.name = "time"
    return df

# Load all sensor data
df_all = load_sensors()

# Ensure index is unique and sorted to avoid get_indexer issues later
df_all = df_all[~df_all.index.duplicated(keep='last')]
df_all = df_all.sort_index()

print(df_all.shape, df_all.index.tz)
df_all.head()

(4119840, 17) UTC


,room,co2_ppm,voc_ppb,pm25_ugm3,awair_score,rh_percent,temp_c,Illumination_lx,spl_a,vector_4d_count,occupancy_count,power_w,energy_kWh,current_a,power_factor,Contact,pir
time,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,225 Olsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 00:01:00+00:00,225 Olsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 00:02:00+00:00,225 Olsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 00:03:00+00:00,225 Olsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 00:04:00+00:00,211 Olsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 4. Label Meetings and Filter Unrealized Meetings

We first:
1. Normalize meeting titles (lowercase, remove punctuation).
2. Assign each event to one of six meeting categories using regex patterns.
3. Save the labeled calendar.
4. Use CO₂ patterns to identify which scheduled meetings actually occurred.

### 4.1 Normalize Meeting Summaries and Apply Regex Classifier

In [ ]:
CAL = df_cal.copy()

def norm_text(x: str) -> str:
    """
    Normalize summary text:
    - Lowercase
    - Remove punctuation (keep basic word chars, spaces, ', &, /, -)
    - Collapse multiple spaces
    """
    if pd.isna(x):
        return ""
    x = re.sub(r"[^\w\s'&/-]+", " ", str(x).lower())
    return re.sub(r"\s+", " ", x).strip()

CAL["summary_norm"] = CAL["summary"].apply(norm_text)

In [ ]:
# Define six main meeting categories with regex rules
CATS = {
    "Admin/Leadership/Programs": [
        r"\b(committee|leadership|governance|program meeting|grant meeting|professional development)\b",
        r"\b(tag ?up|check[- ]?in|coordination|status update|regular)\b",
        r"\b(communications)\b",
        r"\ball hands\b",
    ],
    "Instruction/Student Support": [
        r"\b(office hours|oh)\b",
        r"\b(capstone|undergrad(?!uate)? team|graduate writing lab|reu)\b",
        r"\b(cs|ece)\s?\d{3,5}\b",
        r"\bstudent meeting\b",
        r"\bseminar\b",
    ],
    "Events/Outreach/Social": [
        r"\b(coffee hour|byol|lunch|brown ?bag|mixer|d&d|open house|town hall|workshop)\b",
        r"\b(seminar|guest talk|colloquium)\b",
    ],
    "Research/Lab/Project": [
        r"\b(lab meeting|group meeting|research (group )?meeting|reading group|journal club)\b",
        r"\b(project (meeting|mtg)s?|project\b)\b",
        r"\b(assist|besi[- ]?c|nrt|nzero|m2fed|aracps|cognitiveems|inertia|smart (health|infrastructure))\b",
        r"\b(cavalier autonomous racing|streamflow|hydroinformatics|ria|sif|scwms)\b",
        r"\b(team meeting)\b",
        r"\b(link lab (research|leadership)?|lach group|ben ?calhoun group?)\b",
    ],
    "Walkup": [
        r"\b(walk up meeting)\b",
    ],
    "Other": [
        r"^\b(meeting|1:1|one on one)\b",
        r"^[a-z' ]{2,20}$",  # short single-name meetings
    ],
}

# Compile regex patterns for efficiency
COMPILED = {k: [re.compile(p) for p in v] for k, v in CATS.items()}

def classify(text: str) -> str:
    """
    Classify a meeting summary into one of the six categories using
    regex patterns plus simple fallbacks.
    """
    t = norm_text(text)

    # Explicit ordered patterns (priority order)
    for cat in [
        "Admin/Leadership/Programs",
        "Instruction/Student Support",
        "Events/Outreach/Social",
        "Research/Lab/Project",
        "Walkup",
        "Other",
    ]:
        for rx in COMPILED[cat]:
            if rx.search(t):
                return cat

    # Fallback heuristics
    if "group" in t or "lab" in t or "research" in t:
        return "Research/Lab/Project"
    if "office hours" in t or "student" in t:
        return "Instruction/Student Support"
    if "meeting" in t:
        return "Admin/Leadership/Programs"
    return "Other"

# Apply classifier
CAL["category"] = CAL["summary"].apply(classify)

# Quick category counts
cat_counts = (
    CAL.groupby("category", dropna=False)
       .size()
       .reset_index(name="n_events")
       .sort_values("n_events", ascending=False)
)
display(cat_counts)

,category,n_events
4,Research/Lab/Project,5206
5,Walkup,4095
3,Other,2202
0,Admin/Leadership/Programs,1528
2,Instruction/Student Support,1494
1,Events/Outreach/Social,146


### 4.2 Save Labeled Calendar and Plot Counts

In [ ]:
# Save labeled calendar with categories
ev_labeled_all = CAL.copy()
ev_labeled_all.to_csv(
    EXPORT_DIR / "meetings_labeled_all.csv.gz",
    index=False, compression="gzip"
)
print("Saved labeled calendar →", EXPORT_DIR / "meetings_labeled_all.csv.gz")

# Plot category counts (all scheduled meetings)
plt.figure(figsize=(9, 4))
order = cat_counts.sort_values("n_events", ascending=False)["category"]
sns.countplot(data=CAL, x="category", order=order, palette="viridis")
plt.xticks(rotation=20, ha="right")
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Meeting counts by category (all scheduled meetings)")
savefig("meeting_counts_all")

Saved labeled calendar → exports\meetings_labeled_all.csv.gz
Saved figure → figures\meeting_counts_all.png


### 4.3 $CO_2$-Based Detection of Actual Meetings

In [ ]:
# Before applying the "did the meeting actually happen?" heuristic,
# fill short gaps in sensor data to avoid spurious missingness.
num_cols = df_all.select_dtypes("number").columns
df_all[num_cols] = df_all[num_cols].ffill(limit=10).bfill(limit=10)
df_all[num_cols] = df_all[num_cols].interpolate(
    limit=10, limit_direction="both"
)

def meeting_happened(start, end, df: pd.DataFrame,
                     rise_thresh: float = 10,
                     occupancy_thresh: float = 550,
                     min_points: int = 5) -> bool:
    """
    Heuristic to decide whether a scheduled meeting actually occurred.

    Returns True if:
    - CO₂ rises by >= rise_thresh ppm inside the window, OR
    - Median CO₂ in the window is >= occupancy_thresh.

    Otherwise treat as an unrealized / no-show event.

    Parameters
    ----------
    start, end : Timestamp
        Local (US/Eastern) meeting start and end times.
    df : DataFrame
        Sensor data, indexed by UTC time.
    rise_thresh : float
        Minimum CO₂ rise to flag occupancy.
    occupancy_thresh : float
        Median CO₂ threshold for "already occupied".
    min_points : int
        Minimum number of points required in the window.
    """
    window = df.loc[start:end]
    if len(window) < min_points or window["co2_ppm"].isna().all():
        return False

    co2 = window["co2_ppm"].dropna()
    if co2.empty:
        return False

    # Case 1: clear rise
    if co2.max() - co2.min() >= rise_thresh:
        return True

    # Case 2: continuously elevated (already occupied)
    if co2.median() >= occupancy_thresh:
        return True

    return False

### 4.4 Remove Unrealized Events and Save Counts

In [ ]:
# Apply the meeting_happened heuristic to each labeled event
df_cal = CAL.copy()
df_cal["actual_meeting"] = df_cal.apply(
    lambda row: meeting_happened(row["start"], row["end"], df_all),
    axis=1,
)

# Keep only meetings that likely occurred
df_cal_clean = df_cal[df_cal["actual_meeting"]].reset_index(drop=True)
print(
    f"Dropped {len(df_cal) - len(df_cal_clean)} meetings out of "
    f"{len(df_cal)} that likely did not occur."
)

# Category counts for verified meetings
cat_counts_verified = (
    df_cal_clean["category"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "Category", "category": "Count"})
)
display(cat_counts_verified)

# Save verified counts and plot
cat_counts_verified.to_csv(
    EXPORT_DIR / "meeting_counts_verified.csv", index=False
)

plt.figure(figsize=(8, 4))
order = df_cal_clean["category"].value_counts().index
sns.countplot(
    data=df_cal_clean, x="category", order=order, palette="viridis"
)
plt.xticks(rotation=25, ha="right")
plt.xlabel("Meeting Category")
plt.ylabel("Count of Verified Meetings")
plt.title("Counts of Verified Meetings by Category")
savefig("meeting_counts_verified")

# Verified meetings for downstream analysis
ev_labeled = df_cal_clean.copy()

Dropped 4690 meetings out of 14671 that likely did not occur.


,Count,count
0,Walkup,4048
1,Research/Lab/Project,2988
2,Other,1287
3,Instruction/Student Support,806
4,Admin/Leadership/Programs,748
5,Events/Outreach/Social,104


Saved figure → figures\meeting_counts_verified.png


# 5. Feature Engineering

For each verified meeting we compute:
- Pre-meeting, during-meeting, and post-meeting window summaries.
- Snapshots at meeting end and 30/60 minutes after.
- Impact features (peak deltas and recovery deltas vs pre-meeting baseline).

This produces a feature table `X_all` with one row per meeting.

### 5.1 Window Definitions and Utility Functions

In [ ]:
# Ensure df_all is UTC and sorted (defensive)
df_all = df_all.sort_index()
if df_all.index.tz is None:
    df_all.index = df_all.index.tz_localize("UTC")
else:
    df_all.index = df_all.index.tz_convert("UTC")

# IAQ signals used for impact features (keep only available columns)
signals = [c for c in ["co2_ppm", "voc_ppb", "temp_c",
                       "rh_percent", "pm25_ugm3"]
           if c in df_all.columns]
if not signals:
    raise ValueError(
        f"No known sensor columns found; have: {list(df_all.columns)[:12]}"
    )

@dataclass
class WinCfg:
    """
    Configuration for pre/during/post window lengths and minimum coverage.
    """
    pre_min: int = 30
    post_min: int = 60       # kept for compatibility, not used directly
    min_points: int = 6
    snapshot_tol_min: int = 5  # ± tolerance for snapshot matching

In [ ]:
def slice_stats(ts: pd.DataFrame, cols, label: str) -> dict:
    """
    Compute mean/median/max/min and a simple slope for each column
    in a time window.

    The slope is approximated as the change in the mean of the first
    and last 10% of points divided by window duration (minutes).
    """
    out = {}
    if ts.empty:
        return out

    dt_min = max((ts.index[-1] - ts.index[0]).total_seconds() / 60.0, 1)

    for col in cols:
        s = ts[col].dropna()
        if s.empty:
            continue

        out[f"{label}_{col}_mean"] = s.mean()
        out[f"{label}_{col}_median"] = s.median()
        out[f"{label}_{col}_max"] = s.max()
        out[f"{label}_{col}_min"] = s.min()

        # Simple slope based on first and last 10% means
        first = s.iloc[: max(len(s) // 10, 1)].mean()
        last = s.iloc[-max(len(s) // 10, 1):].mean()
        out[f"{label}_{col}_slope"] = (last - first) / dt_min

    return out

In [ ]:
def snapshot_at(df_all: pd.DataFrame,
                when: pd.Timestamp,
                cols,
                label: str,
                tol_min: int) -> dict:
    """
    Grab the nearest samples to `when` within ± tol_min minutes.

    Returns
    -------
    dict
        {<label>_<col>: value, <label>_ts: timestamp} where available.
    """
    out = {}

    idxer = df_all.index.get_indexer(
        [when],
        method="nearest",
        tolerance=pd.Timedelta(minutes=tol_min)
    )
    pos = idxer[0]
    if pos == -1:
        return out

    row = df_all.iloc[pos]

    for col in cols:
        if col in row and pd.notna(row[col]):
            out[f"{label}_{col}"] = row[col]

    out[f"{label}_ts"] = df_all.index[pos]
    return out

### 5.2 event_impact_features: Pre/During/Post & Impact Deltas

In [ ]:
def event_impact_features(row, cfg: WinCfg = WinCfg()) -> dict | None:
    """
    For one event row with start/end, compute pre, during, and post IAQ features.

    Windows (all in UTC to match df_all):
    - Pre:     [start - pre_min, start]
    - During:  [start, end]
    - Post0_30:   [end, end + 30min]
    - Post30_60:  [end + 30min, end + 60min]

    Requirements:
    - Each of the four windows must have at least cfg.min_points samples.

    Returns
    -------
    dict of features, or None if coverage is insufficient.
    """
    # Convert event times to UTC for alignment with sensors
    t0 = row.start.tz_convert("UTC")
    t1 = row.end.tz_convert("UTC")
    t1_30 = t1 + pd.Timedelta(minutes=30)
    t1_60 = t1 + pd.Timedelta(minutes=60)

    # Extract time windows
    pre = df_all.loc[t0 - pd.Timedelta(minutes=cfg.pre_min): t0]
    during = df_all.loc[t0: t1]
    post0_30 = df_all.loc[t1: t1_30]
    post30_60 = df_all.loc[t1_30: t1_60]

    # Require minimum coverage in each core window
    if any(len(x) < cfg.min_points for x in (pre, during, post0_30, post30_60)):
        return None

    feats: dict[str, float] = {}

    # Window summaries
    feats.update(slice_stats(pre, signals, "pre"))
    feats.update(slice_stats(during, signals, "during"))
    feats.update(slice_stats(post0_30, signals, "post0_30"))
    feats.update(slice_stats(post30_60, signals, "post30_60"))

    # Snapshots at end, +30, +60 minutes
    feats.update(
        snapshot_at(df_all, t1, signals, "post0_snap", cfg.snapshot_tol_min)
    )
    feats.update(
        snapshot_at(df_all, t1_30, signals, "post30_snap", cfg.snapshot_tol_min)
    )
    feats.update(
        snapshot_at(df_all, t1_60, signals, "post60_snap", cfg.snapshot_tol_min)
    )

    # Impact and recovery deltas vs pre-meeting baseline
    for col in signals:
        pre_key = f"pre_{col}_mean"

        # Peak during vs pre mean
        if pre_key in feats and f"during_{col}_max" in feats:
            feats[f"impact_{col}_peak_delta"] = (
                feats[f"during_{col}_max"] - feats[pre_key]
            )

        # Post window means vs pre
        if pre_key in feats and f"post0_30_{col}_mean" in feats:
            feats[f"post0_30_{col}_delta"] = (
                feats[f"post0_30_{col}_mean"] - feats[pre_key]
            )

        if pre_key in feats and f"post30_60_{col}_mean" in feats:
            feats[f"post30_60_{col}_delta"] = (
                feats[f"post30_60_{col}_mean"] - feats[pre_key]
            )

        # Snapshot deltas vs pre
        for snap_label in ("post0_snap", "post30_snap", "post60_snap"):
            snap_key = f"{snap_label}_{col}"
            if pre_key in feats and snap_key in feats:
                feats[f"{snap_label}_{col}_delta"] = (
                    feats[snap_key] - feats[pre_key]
                )

    # Basic metadata
    feats["duration_min"] = (t1 - t0).total_seconds() / 60.0
    feats["start_utc"] = t0
    feats["end_utc"] = t1

    # Category / label (robust to missing)
    mt = getattr(row, "category", None)
    if mt is None or (isinstance(mt, float) and pd.isna(mt)):
        mt = getattr(row, "topic_name", None) or "unknown"
    feats["category"] = mt
    feats["summary"] = getattr(row, "summary", "")

    return feats

### 5.3 Build Full Feature Table `X_all` and Save

In [ ]:
# Build feature table for all verified meetings
rows = []
for r in ev_labeled.itertuples(index=False):
    feats = event_impact_features(r)
    if feats is not None:
        rows.append(feats)

X_all = (
    pd.DataFrame(rows)
    .sort_values("start_utc")
    .reset_index(drop=True)
)

print("Impact feature table:", X_all.shape)
X_all.head()

Impact feature table: (9981, 153)


,pre_co2_ppm_mean,pre_co2_ppm_median,pre_co2_ppm_max,pre_co2_ppm_min,pre_co2_ppm_slope,pre_temp_c_mean,pre_temp_c_median,pre_temp_c_max,pre_temp_c_min,pre_temp_c_slope,...,post30_60_voc_ppb_delta,post0_snap_voc_ppb_delta,post30_snap_voc_ppb_delta,post60_snap_voc_ppb_delta,impact_pm25_ugm3_peak_delta,post0_30_pm25_ugm3_delta,post30_60_pm25_ugm3_delta,post0_snap_pm25_ugm3_delta,post30_snap_pm25_ugm3_delta,post60_snap_pm25_ugm3_delta
0,476.606452,476.0,503.0,457.0,-0.853333,22.932408,23.146667,23.555556,21.760000,0.016751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,472.709677,472.0,493.0,458.0,0.082222,22.932328,23.000001,23.666666,21.760000,0.008479,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,431.554839,427.0,457.0,410.8,-0.386667,22.866660,22.888887,23.255556,22.042222,-0.000780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,441.470968,438.0,459.0,431.0,0.113333,23.904545,24.388890,24.777777,22.080000,0.010000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,431.238710,426.6,448.4,413.8,0.326667,23.864481,24.214999,24.666668,22.080000,-0.054216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Save feature table (wide format)
X_all.to_parquet(EXPORT_DIR / "X_all.parquet", index=False)
X_all.to_csv(EXPORT_DIR / "X_all.csv", index=False)
print("Saved X_all to", EXPORT_DIR)

Saved X_all to exports


# 6. Merge Features with Calendar Metadata

We enrich the feature table with:
- Month, hour, duration from calendar.
- Winter / evening flags.
- Final meeting type label (`group`).

The resulting dataset `Y_all` is used for most of the results in the paper.

### 6.1 Add Calendar Covariates and Rename Category

In [ ]:
CAL2 = ev_labeled.copy()

# Ensure datetime dtype
CAL2["start"] = pd.to_datetime(CAL2["start"])
CAL2["end"] = pd.to_datetime(CAL2["end"])

# Calendar-derived covariates
CAL2["month"] = CAL2["start"].dt.month
CAL2["hour"] = CAL2["start"].dt.hour
CAL2["duration_min_cal"] = (
    (CAL2["end"] - CAL2["start"]).dt.total_seconds() / 60.0
)

# Season / evening flags
CAL2["is_winter"] = CAL2["month"].isin([12, 1, 2])
CAL2["is_evening"] = CAL2["hour"] >= 17

# UTC keys for merging
CAL2["start_utc"] = CAL2["start"].dt.tz_convert("UTC")
CAL2["end_utc"] = CAL2["end"].dt.tz_convert("UTC")

# Rename category -> group for analysis
CAL2 = CAL2.rename(columns={"category": "group"})

### 6.2 Create Final Merged Analysis Dataset `Y_all`

In [ ]:
# Merge IAQ features with calendar covariates
key_cols = ["start_utc", "end_utc", "summary"]

Y_all = X_all.merge(
    CAL2[key_cols + ["group", "month", "hour", "is_winter", "is_evening"]],
    on=key_cols,
    how="left"
)

print("Columns in Y_all:", len(Y_all.columns))
print(Y_all["group"].value_counts(dropna=False))

Columns in Y_all: 158
group
Walkup                         4204
Research/Lab/Project           3018
Other                          1309
Instruction/Student Support     828
Admin/Leadership/Programs       792
Events/Outreach/Social          104
Name: count, dtype: int64


In [ ]:
# Save final analysis dataset
Y_all.to_parquet(EXPORT_DIR / "Y_all.parquet", index=False)
Y_all.to_csv(EXPORT_DIR / "Y_all.csv", index=False)
print("Saved Y_all to", EXPORT_DIR)

Saved Y_all to exports


# 7. Results Part A – Peak Impact by Meeting Type

We summarize `impact_*_peak_delta` metrics by meeting category using:

- Winsorized distributions (box + jitter).
- Mean ± 95% CI plots.
- Kruskal-Wallis tests across categories.
- Pairwise Mann-Whitney tests with Holm correction.

### 7.1 Unified impact_by_category Function

In [ ]:
def impact_by_category(metric: str,
                       X: pd.DataFrame,
                       winsor=(0.01, 0.99)):
    """
    Analyze one impact_*_peak_delta metric using already-merged dataset X.

    Produces:
    - Combined figure (boxplot + mean CI).
    - Kruskal-Wallis test across meeting categories.
    - Pairwise Mann-Whitney tests with Holm correction.

    Parameters
    ----------
    metric : str
        Column name in X (e.g., 'impact_co2_ppm_peak_delta').
    X : DataFrame
        Dataset containing 'group' and the metric.
    winsor : tuple(float, float)
        Lower and upper quantiles for winsorization.

    Returns
    -------
    df_metric : DataFrame
        Winsorized subset with 'group' and metric.
    stats_df : DataFrame
        Pairwise test results with Holm-adjusted p-values.
    """
    if metric not in X.columns:
        raise ValueError(f"Metric '{metric}' not in X.")
    if "group" not in X.columns:
        raise ValueError("X must contain 'group' column.")

    df = X[["group", metric]].dropna().copy()

    # Winsorize to reduce influence of extreme outliers
    lo, hi = df[metric].quantile(winsor)
    df[metric] = df[metric].clip(lo, hi)

    # Order categories by mean metric value
    order = (
        df.groupby("group")[metric]
          .mean()
          .sort_values()
          .index
    )

    # Combined figure (ACM-friendly layout)
    fig, axes = plt.subplots(
        2, 1, figsize=(7, 9), dpi=300,
        gridspec_kw={"height_ratios": [2.2, 1.6]}
    )
    ax1, ax2 = axes

    # ---- 1) Boxplot (top) ----
    sns.boxplot(
        data=df, x="group", y=metric, order=order,
        showfliers=False, ax=ax1, width=0.6
    )
    sns.stripplot(
        data=df, x="group", y=metric,
        order=order, color="k", alpha=0.18, size=1.5, ax=ax1
    )
    ax1.axhline(0, color="0.5", lw=1)
    ax1.set_ylabel(metric.replace("_", " "), fontsize=11)
    ax1.set_xlabel("")
    ax1.set_title("Impact Distribution", fontsize=12, pad=8)

    # Hide x labels on top plot (to avoid clutter)
    ax1.set_xticklabels([])

    ax1.grid(axis="y", color="0.9", linestyle="-", linewidth=0.7)
    ax1.tick_params(axis="y", labelsize=10)

    # ---- 2) Mean ± 95% CI (bottom) ----
    sns.pointplot(
        data=df, x="group", y=metric, order=order,
        estimator=np.mean, errorbar=("ci", 95),
        join=False, capsize=0.2, color="C0", ax=ax2
    )
    ax2.axhline(0, color="0.5", lw=1)
    ax2.set_ylabel("Mean Impact", fontsize=11)
    ax2.set_title("Mean ± 95% CI", fontsize=12, pad=8)
    ax2.grid(axis="y", color="0.9", linestyle="-", linewidth=0.7)

    # Rotate xticklabels so they fit within ACM column width
    ax2.set_xticklabels(
        ax2.get_xticklabels(),
        rotation=35, ha="right", fontsize=9
    )

    plt.suptitle(
        f"{metric.replace('_',' ')} — IAQ Impact by Meeting Category",
        fontsize=12, y=0.98
    )
    plt.tight_layout()

    savefig(f"{metric}_impact_by_category")

    # ---- Statistics: Kruskal–Wallis across categories ----
    groups = [g[metric].values for _, g in df.groupby("group")]
    group_names = list(df.groupby("group").groups.keys())
    H, p_kw = stats.kruskal(*groups, nan_policy="omit")
    print(f"{metric}: Kruskal-Wallis H={H:.3f}, p={p_kw:.3g} "
          f"(k={len(groups)} groups)")

    # ---- Pairwise Mann–Whitney with Holm correction ----
    pairs = []
    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            x, y = groups[i], groups[j]
            U, p = stats.mannwhitneyu(
                x, y, alternative="two-sided"
            )
            n1, n2 = len(x), len(y)
            # Rank-biserial effect size
            r_rb = 1 - (2 * U) / (n1 * n2)
            pairs.append((group_names[i], group_names[j],
                          U, p, n1, n2, r_rb))

    res = pd.DataFrame(
        pairs, columns=["g1", "g2", "U", "p", "n1", "n2", "r_rb"]
    )

    # Holm correction
    res = res.sort_values("p").reset_index(drop=True)
    m = len(res)
    adj = []
    for k, p in enumerate(res["p"], 1):
        adj.append(min((m - k + 1) * p, 1))
    for k in range(1, m):
        adj[k] = max(adj[k], adj[k - 1])
    res["p_holm"] = adj
    res["sig"] = np.where(res["p_holm"] < 0.05, "*", "")

    print("Significant pairwise differences (Holm-corrected):")
    display(res[res["p_holm"] < 0.05])

    # Save stats table
    out_path = STATS_DIR / f"{metric}_pairwise_stats.csv"
    res.to_csv(out_path, index=False)
    print("Saved pairwise stats →", out_path)

    return df, res

### 7.2 Generate Figures for All Impact Metrics

In [ ]:
impact_metrics = [
    "impact_co2_ppm_peak_delta",
    "impact_temp_c_peak_delta",
    "impact_rh_percent_peak_delta",
    "impact_voc_ppb_peak_delta",
    "impact_pm25_ugm3_peak_delta",
]

for m in impact_metrics:
    if m in Y_all.columns:
        df_metric, stats_metric = impact_by_category(m, Y_all)
    else:
        print(f"Metric {m} not found in Y_all; skipping.")

Saved figure → figures\impact_co2_ppm_peak_delta_impact_by_category.png
impact_co2_ppm_peak_delta: Kruskal-Wallis H=93.405, p=1.29e-18 (k=6 groups)
Significant pairwise differences (Holm-corrected):


,g1,g2,U,p,n1,n2,r_rb,p_holm,sig
0,Research/Lab/Project,Walkup,7086381.5,1.946065e-17,3018,4204,-0.117050,2.919098e-16,*
1,Other,Walkup,3030954.0,2.744978e-08,1309,4204,-0.101557,3.842969e-07,*
2,Instruction/Student Support,Walkup,1939444.0,1.910602e-07,828,4204,-0.114331,2.483783e-06,*
3,Admin/Leadership/Programs,Research/Lab/Project,1083284.0,4.920183e-05,792,3018,0.093583,5.904219e-04,*
4,Admin/Leadership/Programs,Instruction/Student Support,298748.0,1.961245e-03,792,828,0.088872,2.157370e-02,*
5,Admin/Leadership/Programs,Other,478117.0,2.821349e-03,792,1309,0.077642,2.821349e-02,*


Saved pairwise stats → stats\impact_co2_ppm_peak_delta_pairwise_stats.csv
Saved figure → figures\impact_temp_c_peak_delta_impact_by_category.png
impact_temp_c_peak_delta: Kruskal-Wallis H=28.180, p=3.36e-05 (k=6 groups)
Significant pairwise differences (Holm-corrected):


,g1,g2,U,p,n1,n2,r_rb,p_holm,sig
0,Other,Walkup,2975693.5,0.000008,1309,4204,-0.081473,0.000124,*
1,Admin/Leadership/Programs,Other,461958.5,0.000028,792,1309,0.108814,0.000398,*
2,Other,Research/Lab/Project,2127479.0,0.000055,1309,3018,-0.077051,0.000719,*


Saved pairwise stats → stats\impact_temp_c_peak_delta_pairwise_stats.csv
Saved figure → figures\impact_rh_percent_peak_delta_impact_by_category.png
impact_rh_percent_peak_delta: Kruskal-Wallis H=45.994, p=9.11e-09 (k=6 groups)
Significant pairwise differences (Holm-corrected):


...

Saved pairwise stats → stats\impact_rh_percent_peak_delta_pairwise_stats.csv
Saved figure → figures\impact_voc_ppb_peak_delta_impact_by_category.png
impact_voc_ppb_peak_delta: Kruskal-Wallis H=47.439, p=4.62e-09 (k=6 groups)
Significant pairwise differences (Holm-corrected):


,g1,g2,U,p,n1,n2,r_rb,p_holm,sig
0,Instruction/Student Support,Walkup,1751430.0,3.267928e-07,768,4087,-0.115981,0.000005,*
1,Events/Outreach/Social,Walkup,262110.0,4.716157e-05,104,4087,-0.233320,0.000660,*
2,Research/Lab/Project,Walkup,5994380.5,2.615533e-04,2789,4087,-0.051771,0.003400,*
3,Admin/Leadership/Programs,Events/Outreach/Social,28904.0,7.441506e-04,699,104,0.204798,0.008930,*
4,Other,Walkup,2752823.0,1.241229e-03,1271,4087,-0.059883,0.013654,*
5,Events/Outreach/Social,Research/Lab/Project,172027.0,1.246360e-03,104,2789,-0.186164,0.013654,*
6,Events/Outreach/Social,Other,78179.0,1.905936e-03,104,1271,-0.182881,0.017153,*


Saved pairwise stats → stats\impact_voc_ppb_peak_delta_pairwise_stats.csv
Saved figure → figures\impact_pm25_ugm3_peak_delta_impact_by_category.png
impact_pm25_ugm3_peak_delta: Kruskal-Wallis H=85.995, p=4.66e-17 (k=6 groups)
Significant pairwise differences (Holm-corrected):


,g1,g2,U,p,n1,n2,r_rb,p_holm,sig
0,Instruction/Student Support,Walkup,1823240.0,1.062153e-12,768,4087,-0.161737,1.593230e-11,*
1,Research/Lab/Project,Walkup,6243246.5,1.698694e-11,2789,4087,-0.095437,2.378172e-10,*
2,Admin/Leadership/Programs,Instruction/Student Support,224352.0,5.407509e-08,699,768,0.164163,7.029762e-07,*
3,Admin/Leadership/Programs,Research/Lab/Project,879624.0,6.448575e-05,699,2789,0.097595,7.738290e-04,*
4,Other,Walkup,2776949.0,1.913713e-04,1271,4087,-0.069172,2.105084e-03,*
5,Instruction/Student Support,Other,534261.0,3.355605e-04,768,1271,-0.094654,3.355605e-03,*
6,Instruction/Student Support,Research/Lab/Project,1145141.0,3.251248e-03,768,2789,-0.069250,2.926123e-02,*


Saved pairwise stats → stats\impact_pm25_ugm3_peak_delta_pairwise_stats.csv


# 8. Results Part B – Recovery Time

We estimate, for each IAQ metric:
- Time for the metric to return within 2% of its pre-meeting baseline.
- Only consider meetings where the metric was above threshold at the end.
- Summarize recovery by metric and meeting category and test for differences.

### 8.1 Compute Recovery Times per Metric

In [ ]:
def metric_recovery_time(start,
                         end,
                         df_all: pd.DataFrame,
                         metric_col: str,
                         baseline_window: str = "15min",
                         recovery_tol: float = 0.02,
                         max_recovery_window: str = "180min"):
    """
    Compute time (in minutes) for a metric to return within
    (1 + recovery_tol) of its pre-meeting baseline mean after meeting end.

    Returns
    -------
    (recovery_minutes, metric_above_threshold_at_end)
    """
    # Baseline before meeting
    baseline_slice = df_all.loc[
        start - pd.Timedelta(baseline_window): start,
        metric_col
    ].dropna()
    if baseline_slice.empty:
        return np.nan, np.nan

    baseline = baseline_slice.mean()
    threshold = baseline * (1 + recovery_tol)

    # After meeting
    after = df_all.loc[
        end: end + pd.Timedelta(max_recovery_window),
        metric_col
    ].dropna()
    if after.empty:
        return np.nan, np.nan

    end_val = df_all.loc[end, metric_col]

    # First time metric falls back below threshold
    recovered = after[after <= threshold]
    if recovered.empty:
        rec_min = np.nan
    else:
        t_recover = recovered.index[0]
        rec_min = (t_recover - end).total_seconds() / 60.0

    return rec_min, end_val > threshold

# Metrics eligible for recovery analysis (must exist in df_all)
metrics_for_recovery = [
    m for m in ["co2_ppm", "temp_c", "rh_percent", "voc_ppb", "pm25_ugm3"]
    if m in df_all.columns
]

# Compute raw recovery times for each verified meeting and metric
recovery_raw = {}

for m in metrics_for_recovery:
    rows = []
    for r in ev_labeled.itertuples(index=False):
        rec_min, above = metric_recovery_time(
            start=r.start,
            end=r.end,
            df_all=df_all,
            metric_col=m,
            baseline_window="15min",
            recovery_tol=0.02,
        )
        rows.append(
            {
                "start": r.start,
                "end": r.end,
                "summary": getattr(r, "summary", None),
                "category": getattr(r, "category", None),
                f"recovery_{m}_2pct": rec_min,
                f"{m}_above_thresh_at_end": above,
            }
        )
    df_rec = pd.DataFrame(rows)
    recovery_raw[m] = df_rec
    df_rec.to_csv(
        EXPORT_DIR / f"recovery_raw_{m}.csv", index=False
    )
    print(f"Saved raw recovery for {m} →",
          EXPORT_DIR / f"recovery_raw_{m}.csv")

Saved raw recovery for co2_ppm → exports\recovery_raw_co2_ppm.csv
Saved raw recovery for temp_c → exports\recovery_raw_temp_c.csv
Saved raw recovery for rh_percent → exports\recovery_raw_rh_percent.csv
Saved raw recovery for voc_ppb → exports\recovery_raw_voc_ppb.csv
Saved raw recovery for pm25_ugm3 → exports\recovery_raw_pm25_ugm3.csv


### 8.2 Filter Nontrivial Cases

In [ ]:
def filter_nontrivial_recovery(df: pd.DataFrame,
                               metric: str,
                               sensor_df: pd.DataFrame) -> pd.DataFrame:
    """
    Keep only meetings where:
    - metric_above_thresh_at_end == True
    - recovery value is non-null
    - metric_end > metric_start (true increase during meeting)
    """
    rec_col = f"recovery_{metric}_2pct"
    above_col = f"{metric}_above_thresh_at_end"

    df2 = df[(df[above_col] == True) & (~df[rec_col].isna())].copy()  # noqa: E712

    # Add metric values at start/end timestamps
    df2[f"{metric}_start"] = df2["start"].map(
        lambda t: sensor_df.loc[t, metric]
    )
    df2[f"{metric}_end"] = df2["end"].map(
        lambda t: sensor_df.loc[t, metric]
    )

    # Keep only meetings where the metric increased
    df2 = df2[df2[f"{metric}_end"] > df2[f"{metric}_start"]]
    return df2

# Build nontrivial sets and save
nontrivial_results = {}
for m in metrics_for_recovery:
    df_nt = filter_nontrivial_recovery(recovery_raw[m], m, df_all)
    nontrivial_results[m] = df_nt
    df_nt.to_csv(
        EXPORT_DIR / f"recovery_nontrivial_{m}.csv", index=False
    )
    print(f"Saved nontrivial recovery for {m} →",
          EXPORT_DIR / f"recovery_nontrivial_{m}.csv")

Saved nontrivial recovery for co2_ppm → exports\recovery_nontrivial_co2_ppm.csv
Saved nontrivial recovery for temp_c → exports\recovery_nontrivial_temp_c.csv
Saved nontrivial recovery for rh_percent → exports\recovery_nontrivial_rh_percent.csv
Saved nontrivial recovery for voc_ppb → exports\recovery_nontrivial_voc_ppb.csv
Saved nontrivial recovery for pm25_ugm3 → exports\recovery_nontrivial_pm25_ugm3.csv


### 8.3 Long-Table Format

In [ ]:
metric_labels = {
    "co2_ppm": "CO",
    "temp_c": "Temperature",
    "rh_percent": "RH",
    "voc_ppb": "VOC",
    "pm25_ugm3": "PM2.5",
}

# Build a long-format table: one row per (meeting, metric)
long_rows = []
for m, df in nontrivial_results.items():
    if m not in metric_labels:
        continue
    rec_col = f"recovery_{m}_2pct"
    for r in df.itertuples(index=False):
        long_rows.append(
            {
                "metric": m,
                "metric_label": metric_labels[m],
                "category": r.category,
                "recovery_min": getattr(r, rec_col),
            }
        )

long_rec = pd.DataFrame(long_rows).dropna(subset=["recovery_min"])
long_rec.to_csv(EXPORT_DIR / "recovery_long.csv", index=False)
print("Saved long-format recovery table →", EXPORT_DIR / "recovery_long.csv")

Saved long-format recovery table → exports\recovery_long.csv


### 8.4 Summary Tables and Kruskal–Wallis Tests

In [ ]:
def summary_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize recovery times by metric and category using:
    count, mean, median, std, 25th and 75th percentiles.
    """
    return (
        df.groupby(["metric_label", "category"])["recovery_min"]
          .agg(
              ["count", "mean", "median", "std",
               lambda x: x.quantile(0.25),
               lambda x: x.quantile(0.75)]
          )
          .rename(columns={"<lambda_0>": "q25", "<lambda_1>": "q75"})
          .sort_values(["metric_label", "mean"])
    )

summary_rec = summary_table(long_rec)
display(summary_rec)

summary_rec.to_csv(
    STATS_DIR / "recovery_summary_by_metric_category.csv"
)
print("Saved recovery summary →",
      STATS_DIR / "recovery_summary_by_metric_category.csv")

# Boxplot of all metrics together
plt.figure(figsize=(12, 5))
sns.boxplot(
    data=long_rec,
    x="category",
    y="recovery_min",
    hue="metric_label",
    showfliers=False,
)
sns.stripplot(
    data=long_rec,
    x="category",
    y="recovery_min",
    hue=None,
    color="k",
    size=2,
    alpha=0.2,
)
plt.xticks(rotation=25, ha="right")
plt.ylabel("Recovery time (min, 2% baseline)")
plt.title("Post-meeting recovery by metric and category")
plt.legend(title="Metric")
savefig("recovery_all_metrics_by_category")

# Kruskal–Wallis per metric across categories
print("\nKruskal-Wallis tests across meeting categories (recovery):\n")

kw_rows = []
for m, label in metric_labels.items():
    dfm = long_rec[long_rec["metric"] == m]
    samples = [
        g["recovery_min"].dropna().values
        for _, g in dfm.groupby("category")
    ]
    names = list(dfm.groupby("category").groups.keys())
    if len(samples) >= 2 and all(len(s) >= 5 for s in samples):
        H, p = stats.kruskal(*samples, nan_policy="omit")
        kw_rows.append({"metric": label, "H": H, "p": p,
                        "k": len(samples)})
        print(f"{label:10s}: H = {H:.3f}, p = {p:.3g} "
              f"({len(samples)} groups)")
    else:
        print(f"{label:10s}: Not enough data per category")

kw_df = pd.DataFrame(kw_rows)
kw_df.to_csv(STATS_DIR / "recovery_kw_tests.csv", index=False)
print("Saved KW tests →", STATS_DIR / "recovery_kw_tests.csv")

count       mean  median        std  \
metric_label category                                                           
CO           Research/Lab/Project          1078  15.733766     3.0  30.265930   
             Admin/Leadership/Programs      269  15.910781     4.0  28.626641   
             Walkup                        1532  17.027415     3.0  32.932078   
             Other                          499  17.525050     3.0  34.807458   
             Events/Outreach/Social          40  18.650000     2.5  37.306458   
             Instruction/Student Support    304  22.424342     3.0  39.282268   
PM2.5        Walkup                        1275   3.614902     2.0   7.807532   
             Instruction/Student Support    228   3.657895     2.0   4.629591   
             Other                          406   4.123153     2.0   9.827380   
             Research/Lab/Project           902   4.565410     2.0   9.171062   
             Events/Outreach/Social          36   4.833333     1.0   8.279061   
             Admin/Leadership/Programs      203   4.842365     2.0   9.745513   
RH           Events/Outreach/Social          29   3.655172     2.0   4.236541   
             Walkup                        1158   5.557858     2.0  17.090577   
             Other                          379   5.997361     2.0  14.605301   
             Admin/Leadership/Programs      222   6.427928     2.0  20.838785   
             Research/Lab/Project           910   6.510989     2.0  19.093004   
             Instruction/Student Support    234   9.790598     2.0  25.955123   
Temperature  Events/Outreach/Social          37   3.459459     2.0   5.403675   
             Other                          392   3.813776     2.0  10.156752   
             Research/Lab/Project           925   3.818378     2.0   9.306092   
             Walkup                        1199   3.868224     2.0  10.228409   
             Admin/Leadership/Programs      251   4.585657     2.0  11.395246   
             Instruction/Student Support    273   4.783883     2.0  13.908346   
VOC          Other                          441   8.551020     2.0  21.862437   
             Walkup                        1438   8.574409     2.0  21.341175   
             Research/Lab/Project           995  10.743719     2.0  25.733264   
             Instruction/Student Support    276  10.771739     2.0  26.396599   
             Admin/Leadership/Programs      234  12.952991     2.0  29.253699   
             Events/Outreach/Social          40  14.425000     3.0  27.136774   

                                          q25    q75  
metric_label category                                 
CO           Research/Lab/Project         1.0  12.00  
             Admin/Leadership/Programs    1.0  16.00  
             Walkup                       1.0  11.00  
             Other                        1.0  12.00  
             Events/Outreach/Social       1.0  10.25  
             Instruction/Student Support  1.0  23.25  
PM2.5        Walkup                       1.0   3.00  
             Instruction/Student Support  1.0   4.00  
             Other                        1.0   4.00  
             Research/Lab/Project         1.0   4.00  
             Events/Outreach/Social       1.0   5.00  
             Admin/Leadership/Programs    1.0   4.00  
RH           Events/Outreach/Social       1.0   5.00  
             Walkup                       1.0   3.00  
             Other                        1.0   4.00  
             Admin/Leadership/Programs    1.0   4.00  
             Research/Lab/Project         1.0   4.00  
             Instruction/Student Support  1.0   4.00  
Temperature  Events/Outreach/Social       1.0   4.00  
             Other                        1.0   3.00  
             Research/Lab/Project         1.0   3.00  
             Walkup                       1.0   3.00  
             Admin/Leadership/Programs    1.0   4.00  
             Instruction/Student Support  1.0   4.00  
VOC          Other 

Saved recovery summary → stats\recovery_summary_by_metric_category.csv
Saved figure → figures\recovery_all_metrics_by_category.png

Kruskal-Wallis tests across meeting categories (recovery):

CO        : H = 6.933, p = 0.226 (6 groups)
Temperature: H = 3.131, p = 0.68 (6 groups)
RH        : H = 10.137, p = 0.0714 (6 groups)
VOC       : H = 6.381, p = 0.271 (6 groups)
PM2.5     : H = 13.533, p = 0.0189 (6 groups)
Saved KW tests → stats\recovery_kw_tests.csv


# 9. Results Part C – Duration Effects

We examine how CO₂ peak impact varies jointly with:
- Meeting type (`group`).
- Duration bins: Short (<30), Medium (30–60), Long (>60).

In [ ]:
# Define duration bins
bins = [0, 30, 60, np.inf]
labels = ["Short (<30)", "Medium (30-60)", "Long (>60)"]

Y_all["duration_bin"] = pd.cut(
    Y_all["duration_min"], bins=bins,
    labels=labels, right=True
)

# Boxplot of CO₂ impact by group and duration bin
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=Y_all,
    x="group",
    y="impact_co2_ppm_peak_delta",
    hue="duration_bin",
)
plt.xticks(rotation=45, ha="right")
plt.ylabel("$CO_2$ peak delta (ppm)")
plt.title("$CO_2$ impact vs meeting type and duration (all months)")
savefig("co2_impact_by_type_and_duration_all")

Saved figure → figures\co2_impact_by_type_and_duration_all.png


In [ ]:
# Scatter plot and trend lines (lmplot) for CO₂ impact vs duration

plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=Y_all,
    x="duration_min",
    y="impact_co2_ppm_peak_delta",
    hue="group",
    alpha=0.5,
)
plt.xlabel("Meeting duration (min)")
plt.ylabel("$CO_2$ peak delta (ppm)")
plt.title("$CO_2$ impact vs duration by meeting type (all months)")
savefig("co2_impact_vs_duration_scatter")

sns.lmplot(
    data=Y_all,
    x="duration_min",
    y="impact_co2_ppm_peak_delta",
    hue="group",
    scatter_kws={"alpha": 0.3},
    height=6,
    aspect=1.4,
)
plt.xlabel("Meeting duration (min)")
plt.ylabel("$CO_2$ peak delta (ppm)")
plt.title("$CO_2$ impact vs duration by meeting type (with trend lines)")
savefig("co2_impact_vs_duration_lm")

Saved figure → figures\co2_impact_vs_duration_scatter.png
Saved figure → figures\co2_impact_vs_duration_lm.png


# 10. Results Part D – Time-Course Traces

We compute mean ΔCO₂ and ΔVOC time-courses relative to meeting start:

- Baseline: mean level from −30 to 0 minutes.
- Δ metric = value − baseline.
- Average across all meetings within each category.

This yields smoothed curves for each meeting type.

### 10.1 CO₂ Time-Course (ΔCO₂)

In [ ]:
# Use full-resolution CO₂ series (no down-sampling)
co2_global = df_all["co2_ppm"][::].dropna()
co2_times = co2_global.index
co2_vals = co2_global.values

def mean_co2_change_timecourse_ultra(CAL2, pre=30, post=60):
    """
    Compute and plot mean change in CO₂ (Δ from baseline) by meeting category
    over [-pre, +post] minutes relative to meeting start.
    """
    rows = []
    for ev in CAL2.itertuples(index=False):
        start = ev.start_utc

        # Relative time vector (minutes)
        rel_min = (co2_times - start).total_seconds() / 60.0
        mask = (rel_min >= -pre) & (rel_min <= post)
        if not mask.any():
            continue

        rel_slice = rel_min[mask]
        co2_slice = co2_vals[mask]

        # Baseline = average CO₂ from −pre to 0 minutes
        base_mask = (rel_slice >= -pre) & (rel_slice <= 0)
        if not base_mask.any():
            continue

        baseline = co2_slice[base_mask].mean()
        delta = co2_slice - baseline

        rows.append(pd.DataFrame({
            "rel_min": rel_slice,
            "delta_co2": delta,
            "group": ev.group
        }))

    df_tc = pd.concat(rows, ignore_index=True)
    df_tc["rel_min"] = df_tc["rel_min"].round().astype(int)

    # Mean ΔCO₂ per group per time bin
    df_mean = (
        df_tc.groupby(["group", "rel_min"])["delta_co2"]
             .mean()
             .reset_index()
    )

    # Plot time-courses with light smoothing
    plt.figure(figsize=(10, 5))
    for g in sorted(df_mean["group"].unique()):
        gdf = df_mean[df_mean["group"] == g].sort_values("rel_min")
        gdf["smooth"] = gdf["delta_co2"].rolling(
            5, center=True, min_periods=1
        ).mean()
        plt.plot(gdf["rel_min"], gdf["smooth"], label=g)

    plt.axvline(0, linestyle="--", color="k")
    plt.xlabel("Minutes relative to meeting start")
    plt.ylabel("ΔCO from baseline (ppm)")
    plt.title("Mean Change in $CO_2$ Over Time by Meeting Category")
    plt.legend()
    savefig("mean_delta_co2_timecourse_ultra")
    plt.show()

# Run CO₂ time-course analysis
mean_co2_change_timecourse_ultra(CAL2)

Saved figure → figures\mean_delta_co2_timecourse_ultra.png


### 10.2 VOC Time-Course (ΔVOC)

In [ ]:
voc_global = df_all["voc_ppb"][::].dropna()
voc_times = voc_global.index
voc_vals = voc_global.values

def mean_voc_change_timecourse_ultra(CAL2, pre=30, post=60):
    """
    Compute and plot mean change in VOC (Δ from baseline) by meeting category
    over [-pre, +post] minutes relative to meeting start.
    """
    rows = []
    for ev in CAL2.itertuples(index=False):
        start = ev.start_utc

        # Relative time vector (minutes)
        rel_min = (voc_times - start).total_seconds() / 60.0
        mask = (rel_min >= -pre) & (rel_min <= post)
        if not mask.any():
            continue

        rel_slice = rel_min[mask]
        voc_slice = voc_vals[mask]

        # Baseline VOC = −pre to 0 minutes
        base_mask = (rel_slice >= -pre) & (rel_slice <= 0)
        if not base_mask.any():
            continue

        baseline = voc_slice[base_mask].mean()
        delta = voc_slice - baseline

        rows.append(pd.DataFrame({
            "rel_min": rel_slice,
            "delta_voc": delta,
            "group": ev.group
        }))

    df_tc = pd.concat(rows, ignore_index=True)
    df_tc["rel_min"] = df_tc["rel_min"].round().astype(int)

    # Mean ΔVOC per group per time bin
    df_mean = (
        df_tc.groupby(["group", "rel_min"])["delta_voc"]
             .mean()
             .reset_index()
    )

    # Plot time-courses with smoothing
    plt.figure(figsize=(10, 5))
    for g in sorted(df_mean["group"].unique()):
        gdf = df_mean[df_mean["group"] == g].sort_values("rel_min")
        gdf["smooth"] = gdf["delta_voc"].rolling(
            5, center=True, min_periods=1
        ).mean()
        plt.plot(gdf["rel_min"], gdf["smooth"], label=g)

    plt.axvline(0, linestyle="--", color="k")
    plt.xlabel("Minutes relative to meeting start")
    plt.ylabel("ΔVOC from baseline (ppb)")
    plt.title("Mean Change in VOC Over Time by Meeting Category")
    plt.legend()
    savefig("mean_delta_voc_timecourse_ultra")
    plt.show()

# Run VOC time-course analysis
mean_voc_change_timecourse_ultra(CAL2)

Saved figure → figures\mean_delta_voc_timecourse_ultra.png
